In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Spark 세션 생성
spark = SparkSession.builder.appName("example").master("spark://neivekim76.local:7077").getOrCreate()

23/12/28 17:36:46 WARN Utils: Your hostname, neivekim76.local resolves to a loopback address: 127.0.0.1; using 172.30.1.27 instead (on interface en0)
23/12/28 17:36:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/28 17:36:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# 예제 데이터 생성 (데이터프레임으로 가정)
data = [(1.0, 2.0, 3.0), (4.0, 5.0, 6.0), (7.0, 8.0, 9.0), (7.0, 8.0, 9.0), (7.0, 8.0, 9.0), (7.0, 8.0, 9.0), (7.0, 8.0, 9.0) ,(7.0, 8.0, 2.0), (7.0, 0.0, 9.0)
        , (3.0, 8.0, 9.0), (5.0, 8.0, 9.0), (7.0, 3.0, 9.0), (7.0, 0.0, 2.0), (7.0, 8.0, 6.0), (5.0, 8.0, 9.0), (7.0, 8.0, 9.0), (7.0, 8.0, 9.0), (0.0, 8.0, 9.0)
        , (3.0, 8.0, 9.0), (9.0, 8.0, 9.0), (7.0, 8.0, 1.0), (1.0, 8.0, 9.0), (7.0, 8.0, 7.0), (7.0, 8.0, 9.0)]
columns = ["feature1", "feature2", "label"]
df = spark.createDataFrame(data, columns)

In [9]:
# 데이터를 훈련 및 테스트 세트로 나누기
train_data, test_data = df.randomSplit([0.8, 0.2], seed=123)

In [10]:
# VectorAssembler: Feature 컬럼 생성
feature_cols = ["feature1", "feature2"]
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# StandardScaler: Feature 컬럼 표준화
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

# RandomForestRegressor: 회귀 모델
rf_regressor = RandomForestRegressor(featuresCol="scaled_features", labelCol="label")

# 파이프라인 생성
pipeline = Pipeline(stages=[vector_assembler, scaler, rf_regressor])

In [11]:
# 파이프라인 모델 학습
model = pipeline.fit(train_data)

23/12/28 17:40:26 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 32 to 21 (= number of training instances)


In [12]:
# 테스트 데이터에 대한 예측
predictions = model.transform(test_data)

# 예측 결과 출력
predictions.show()

+--------+--------+-----+---------+--------------------+-----------------+
|feature1|feature2|label| features|     scaled_features|       prediction|
+--------+--------+-----+---------+--------------------+-----------------+
|     7.0|     3.0|  9.0|[7.0,3.0]|[0.60894413862755...|6.697559523809524|
|     7.0|     8.0|  9.0|[7.0,8.0]|[0.60894413862755...| 7.55135334436805|
|     7.0|     8.0|  7.0|[7.0,8.0]|[0.60894413862755...| 7.55135334436805|
+--------+--------+-----+---------+--------------------+-----------------+

